# Les décorateurs
## Explications

In [21]:
def tartine(garniture):
    def wrapper(*args, **kwargs):
        price = garniture(*args, **kwargs)
        print("\\________/")

        return price + 1

    return wrapper


def sandwich(garniture):
    def wrapper(*args, **kwargs):
        print("/TTTTTTTT\\")
        price = garniture(*args, **kwargs)
        print("\\________/")

        return price + 1

    return wrapper

@sandwich
def kebab(salade_tomates_oignins:bool):
    if salade_tomates_oignins:
        print("salade_tomates_oignins")
    print('viande')

    return 5

@sandwich
def parisien():
    print("  Jambon ")
    print("  Beurre ")

    return 6

@sandwich
def bagnat():
    print("  thon ")
    print("  salade ")

    return 7


In [22]:
kebab(True)

/TTTTTTTT\
salade_tomates_oignins
viande
\________/


6

In [15]:
print(parisien())
print(parisien.__name__)

/TTTTTTTT\
  Jambon 
  Beurre 
\________/
None
wrapper


In [11]:
bagnat()

/TTTTTTTT\
  thon 
  salade 
\________/


In [ ]:
sandwich(parisien)()

## Exercices
### Exercice 1

In [40]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        value = func(*args, **kwargs)
        end = time.time()
        print(end - start, "secondes se sont écoulées")
        return value

    return wrapper

@time_it
def hard_work():
    for x in range(1_000_000):
        y = x ** 2


In [26]:
hard_work()

0.043501853942871094 secondes se sont écoulées


### Second exercice

In [28]:
def count_calls(func):
    counts = 0
    def wrapper(*args, **kwargs):
        nonlocal counts
        counts += 1
        return func(*args, **kwargs)

    def calls():
        return counts

    wrapper.nbcalls = calls

    return wrapper

@count_calls
def some_func():
    print("func called")

@count_calls
def get_data(value):
    return value * 2

print(some_func.nbcalls())
some_func()
some_func()
print(some_func.nbcalls())

0
func called
func called
2


### Troisième exercice

In [41]:
import time
from collections import OrderedDict

def long_calls(func):

    previouses = OrderedDict()

    def inner(value:int):
        try:
            result = previouses[value]
            previouses.move_to_end(value)
        except KeyError:
            result = func(value)
            previouses[value] = result
            if len(previouses) > 5:
                previouses.popitem(last=False)

        return result

    return inner

@time_it
@long_calls
def process(value):
    time.sleep(2)
    return value**2


In [42]:
print(process(1))
print(process(2))
print(process(3))
print(process(4))
print(process(5))
print(process(6))
print(process(1))
print(process(3))


2.00510311126709 secondes se sont écoulées
1
2.003181219100952 secondes se sont écoulées
4
2.005270004272461 secondes se sont écoulées
9
2.005157947540283 secondes se sont écoulées
16
2.0048410892486572 secondes se sont écoulées
25
2.002764940261841 secondes se sont écoulées
36
2.0051159858703613 secondes se sont écoulées
1
5.9604644775390625e-06 secondes se sont écoulées
9


### Exercice 4

In [1]:
def register(notification):
    targets.append(notification)
    return notification

targets = []

In [2]:
@register
def notify_mail():
    print("Envoi mail")

def notify_SMS():
    print("Envoi SMS")

@register
def notify_push():
    print("Push information")

In [3]:

def send_notifications():
    for func in targets:
        func()

send_notifications()

Envoi mail
Push information


In [4]:
notify_mail()

TypeError: 'NoneType' object is not callable